<!-- vscode-jupyter-toc -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->
<a id='toc0_'></a>**Содержание**    
- [Исполняемые файлы](#toc1_)    
  - [Первый процесс](#toc1_1_)    
  - [Исполняемые файлы](#toc1_2_)    
  - [Заголовок исполняемого файла](#toc1_3_)    
  - [Заголовок ELF файла](#toc1_4_)    
    - [Например:](#toc1_4_1_)    
  - [Точка входа](#toc1_5_)    
  - [Описание адресного пространства](#toc1_6_)    
  - [Типичное адресное пространство (первое приближение)](#toc1_7_)    
  - [Типичное адресное пространство (второе приближение)](#toc1_8_)    
  - [ELF programm headers](#toc1_9_)    
    - [На примере того же main.c](#toc1_9_1_)    
  - [Загрузка исполняемого файла](#toc1_10_)    
- [Библиотеки](#toc2_)    
  - [Динамические библиотеки](#toc2_1_)    
  - [Компоновщик](#toc2_2_)    
  - [Динамический компоновщик](#toc2_3_)    
  - [Загрузка ELF файла с динамическими библиотеками](#toc2_4_)    
  - [Поиск динамических библиотек](#toc2_5_)    
  - [Редактирование связей](#toc2_6_)    
- [Как это работает в случае ELF файлов](#toc3_)    
  - [GOT (для переменных)](#toc3_1_)    
  - [PLT (для процедур/функций)](#toc3_2_)    
- [Системные вызовы](#toc4_)    
  - [Системные вызовы](#toc4_1_)    
  - [Реализация системных вызовов](#toc4_2_)    
  - [И сново о прерываниях...](#toc4_3_)    
  - [Программные прерывания](#toc4_4_)    
  - [Программные прерывания в x86](#toc4_5_)    
  - [Дескриптор IDT, поле DPL (desrciptor privilege level)](#toc4_6_)    
    - [Тут нужно отметить](#toc4_6_1_)    
  - [Стек обработчика прерывания](#toc4_7_)    
  - [Отдельный стек для ядра](#toc4_8_)    
  - [Task State Segment (х86)](#toc4_9_)    
  - [Резюме](#toc4_10_)    
  - ["Прыжок" в userspace](#toc4_11_)    
- [Средства управления процессами](#toc5_)    
  - [Системные вызовы](#toc5_1_)    
  - [Создание процессов](#toc5_2_)    
  - [Уничтожение процессов](#toc5_3_)    
  - [Запуск исполняемых файлов](#toc5_4_)    
  - [Файловые дескрипторы](#toc5_5_)    
  - [Дублирование дескрипторов](#toc5_6_)    
- [IPC](#toc6_)    
  - [Каналы (Pipes)](#toc6_1_)    
  - [Именованные каналы (FIFO)](#toc6_2_)    
  - [Сегменты разделяемой памяти (по смыслу - как файл, только в RAM)](#toc6_3_)    
  - [Process Trace](#toc6_4_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- /vscode-jupyter-toc -->

# <a id='toc1_'></a>[Исполняемые файлы](#toc0_)

## <a id='toc1_1_'></a>[Первый процесс](#toc0_)

Ядро ОС настроило прерывания, аллокаторы,планировщик. Что дальше?
    
    мы должны запустить первый процесс и первое приложение!
    например, Linux проверяет (пытается запустить один из) файлы: /sbin/init,/etc/init, /bin/init и /bin/sh.

## <a id='toc1_2_'></a>[Исполняемые файлы](#toc0_)

Существует множество форматов исполняемыхфайлов:
    
    ELF, a.out, PE, COM, Mach-O;
    скрипты, начинающиеся с #! (sha-bang).

## <a id='toc1_3_'></a>[Заголовок исполняемого файла](#toc0_)

    Заголовок:Imagic number/string - позволяет быстро определить формат файла;
    различного рода флаги и параметры:
        версия формата исполняемого файла;
        архитектура;
        ссылки на другие части файла (адрес, где в файле код, где данные и др.).

## <a id='toc1_4_'></a>[Заголовок ELF файла](#toc0_)

	struct elf_hdr {
		uint8_t e_ident[ELF_NIDENT];    // [16]
		uint16_t e_type;				// lib or exec
		uint16_t e_machine;				// arch
		uint32_t e_version;

		/* Logical addr of the first instriction */
		uint64_t e_entry;
		
		uint64_t e_phoff;
		uint64_t e_shoff;
		uint32_t e_flags;
		uint16_t e_ehsize;
		uint16_t e_phentsize;
		uint16_t e_phnum;
		uint16_t e_shentsize;
		uint16_t e_shnum;
		uint16_t e_shstrndx;
	} __attribute__((packed));

### <a id='toc1_4_1_'></a>[Например:](#toc0_)

    #include <stdio.h>

    int main()
    {
        printf("Hello, World!\n");
        printf("main address: %p\n", &main);   // адрес функции main (по идее точка входа)
        return 0;
    }

Hello, World!  
main address: 0x556801ac3169

`$ readelf -h /home/user1/projects/stepik.org/OS/5.Userspace/main`

    Заголовок ELF:
        Magic:   7f 45 4c 46 02 01 01 00 00 00 00 00 00 00 00 00 
        Класс:                             ELF64
        Данные:                            дополнение до 2, от младшего к старшему
        Version:                           1 (current)
        OS/ABI:                            UNIX - System V
        Версия ABI:                        0
        Тип:                               DYN (Совм. исп. объектный файл)
        Машина:                            Advanced Micro Devices X86-64
        Версия:                            0x1
        Адрес точки входа:               0x1080 (ВООБЩЕ не совпадает, значит до main() еще что-то исполняется, например обработчики параметров командной строки или еще что-то)
        Начало заголовков программы:          64 (байт в файле)
        Начало заголовков раздела:          17024 (байт в файле)
        Флаги:                             0x0
        Size of this header:               64 (bytes)
        Size of program headers:           56 (bytes)
        Number of program headers:         13
        Size of section headers:           64 (bytes)
        Number of section headers:         36
        Section header string table index: 35 

## <a id='toc1_5_'></a>[Точка входа](#toc0_)

У любой программы есть первая инструкция - точка входа:

    формат исполняемого файла явно или не явно указывает адрес первой инструкции;
    ОС после загрузки исполняемого файла передает управление первой инструкции;
    обычно передача управления сопровождается понижением уровня привилегий кода (переходом в userspace).

## <a id='toc1_6_'></a>[Описание адресного пространства](#toc0_)

Формат исполняемого файла описывает логическое адресное пространство:

    какие участки логического адресного пространства нужны и для чего;
    где в памяти процесса должны располагаться код и данные;
    где в исполняемом файле хранятся код и данные.
    


## <a id='toc1_7_'></a>[Типичное адресное пространство (первое приближение)](#toc0_)

x86-64 память от 0 до 2\*\*64

    Kernelspace     *** -> 0xfff800000000000 (тут код ядра, данные ядра)

    Canonical Hole  0xfff800000000000-0х1 - 0х8000000000000000

    Userspace       0х8000000000000000-0x1 - 0x0

В х86-32 (до 4Гб) проще - дыры нет, делилось либо первые 2Гб kernel, остальное user (вариант Windows), либо 1Гб kernel, остальное user (Linux по умолчанию).

Зачем вообще нужен kernel space в логическом адресном пространстве программы? Почему бы вообще не показывать его приложениям в принципе?    

Представим, что код приложения хочет обратиться к сервису ядра (или просто происходит перывание), он должен сделать системный вызов, и реализация системного вызова должна передать управление ядру. Если бы ядро пользовалось своей собственной таблицей страниц независимой от таблиц страниц приложений, то при системном вызове было бы необходимо подменять указатель на корень таблицы страниц, чтобы передать управление коду ядра.  
1. в x86 до некоторого времени нельзя было организовать такое переключение  
2. переключение корня таблицы страниц означет сброс TLB, а это потеря производительности  
3. механизм таблиц страниц позволяет защитить привелигированные участки памяти от действий непривелигированного кода, а значит и поддерживать отдельную таблицу страниц для ядра нет особого смысла  


## <a id='toc1_8_'></a>[Типичное адресное пространство (второе приближение)](#toc0_)

В файле приложения частично описано адресное пространство, которым оно будет пользоваться.  

    Kernalspace
    -> *empty*
    
    Stack       // стек растет ВНИЗ, поэтому его ставят наверх (обычно в приложении не описывается, но можно и описать)
                   обычно создается ОС и является способом передачи приложению каких-то параметров (напр. арг. командной 
                   строки - ОС выделяет стек, сохр. там арг. строки, выставляет указатель стека, приложение может пойти выше 
                   указателя и забрать там эти параметры, которые подготовила ему ОС)
    -> *empty*  // (место под рост стека)
    mmap/so/dll // где-то между кучей и стеком в лог.адр.прост. приложения располагаются разделяемые библиотеки. В исполняемом
                   файле только указание, какие нужны. ОС размещает их по своему алгоритму в адр.прост. приложения.
    -> *empty*  // (место под рост кучи)
                //  куча (дин.память) растет ВВЕХ. Обычно также не описывается в файле приложения, а предоставляется ОС и 
                    может увеличиваться в помощью системных вызовов.
    Heap
    -> *empty*  // (обычно небольшой пропуск или просто выравнивание)
    Data        // глобальные и статические переменные (только проинициализированные)
    Code/text   // непосредственно инструкции процессора
    -> *empty*  // (до конца памяти)
    0x0         // обычно код никогда не пишется до 0х0, именно поэтому разыменование 0x0 -> SEGFAULT

*) где тут будет bss (непроинициализированные статические переменные, т.е. просто аллокация под будущие данные) ? Наверно внизу кучи? или как часть data? 

## <a id='toc1_9_'></a>[ELF programm headers](#toc0_)

В формате ELF за описание адресного пространства приложения, того чем его надо заполнить отвечает programm headers.  

	struct elf64_hdr { // в заголовке elf файла хранится информация о заголовках программы
        ...
        /* Offset of the programm header table */		
		uint64_t e_phoff;		// где начинается таблица

		uint64_t e_shoff;
		uint32_t e_flags;
		uint16_t e_ehsize;

        /* The size of a programm header table entry */
		uint16_t e_phentsize;
        /* The number of entries in the programm header table*/
		uint16_t e_phnum;		// сколько записей в таблице
        
		uint16_t e_shentsize;
		uint16_t e_shnum;
		uint16_t e_shstrndx;
	} __attribute__((packed));

	/* формат записи в таблице programm headers */
	struct elf64_phdr {
		uint32_t p_type;	// тип записи (нас интересует из всех типов только PT_LOAD == 1, 
							   которые соответствуют описанию участков лог.адр.прост. 
							   (еще говорят - описанию сегментов)
		uint32_t p_flags;	// можно ли читать/писать/исполнять код из этого сегмента
		uint64_t p_offset;	// смещение в ФАЙЛЕ, где лежат данные, которые надо скопировать в сегмент
		uint64_t p_vaddr;	// лог. и ...
		uint64_t p_paddr;	// физ. адрес данного участка (сегмента) в памяти (начало)
		uint64_t p_filesz;	// размер в файле начальных данных для сегмента (м.б. меньше участка, хвост заполняется нулями)
		uint64_t p_memsz;	// размер участка (сегмента), куда копировать из файла начальные данные
		uint64_t p_align;
	} __attribute__((packed));

\*) если в таком заголовке будет указано, скопировать что-то в область ядра, это будет задача ОС отследить и прекратить исполнение такого файла. Причем руту по доступу к памяти никаких привелегий не дается.  
\*\*) в случае модулей ядра запись в kernelspace (хз производится по факту или нет), но они загружаются системным вызовом, т.е. ядро само определяет как должна работать эта процедура, поэтому бесконтрольной записи в kernelspace быть не может.

### <a id='toc1_9_1_'></a>[На примере того же main.c](#toc0_)

`$ readelf -l /home/user1/projects/stepik.org/OS/5.Userspace/main`

    Тип файла ELF — DYN (Совм. исп. объектный файл)
    Точка входа 0x1080
    Имеется 13 заголовков программы, начиная со смещения 64

    Заголовки программы:
      Тип            Смещ.              Вирт.адр           Физ.адр
                    Рзм.фйл            Рзм.пм              Флаги  Выравн
      PHDR           0x0000000000000040 0x0000000000000040 0x0000000000000040
                    0x00000000000002d8 0x00000000000002d8  R      0x8
      INTERP         0x0000000000000318 0x0000000000000318 0x0000000000000318
                    0x000000000000001c 0x000000000000001c  R      0x1
          [Запрашиваемый интерпретатор программы: /lib64/ld-linux-x86-64.so.2]    // типа LOAD получилось 4 штуки (в примере на видео 2 штуки ???)
      LOAD           0x0000000000000000 0x0000000000000000 0x0000000000000000     // и какого же тут 0x0 ?? *
                    0x0000000000000638 0x0000000000000638  R      0x1000          // зачем это и ниже ХЗ? *
      LOAD           0x0000000000001000 0x0000000000001000 0x0000000000001000
                    0x0000000000000225 0x0000000000000225  R E    0x1000          // read/exec - очевидно тут лежит код
      LOAD           0x0000000000002000 0x0000000000002000 0x0000000000002000
                    0x0000000000000170 0x0000000000000170  R      0x1000
      LOAD           0x0000000000002db0 0x0000000000003db0 0x0000000000003db0
                    0x0000000000000260 0x0000000000000268  RW     0x1000          // read/write - очевидно тут лежат данные
      DYNAMIC        0x0000000000002dc0 0x0000000000003dc0 0x0000000000003dc0
                    0x00000000000001f0 0x00000000000001f0  RW     0x8
      NOTE           0x0000000000000338 0x0000000000000338 0x0000000000000338
                    0x0000000000000020 0x0000000000000020  R      0x8
      NOTE           0x0000000000000358 0x0000000000000358 0x0000000000000358
                    0x0000000000000044 0x0000000000000044  R      0x4
      GNU_PROPERTY   0x0000000000000338 0x0000000000000338 0x0000000000000338
                    0x0000000000000020 0x0000000000000020  R      0x8
      GNU_EH_FRAME   0x0000000000002024 0x0000000000002024 0x0000000000002024
                    0x0000000000000044 0x0000000000000044  R      0x4
      GNU_STACK      0x0000000000000000 0x0000000000000000 0x0000000000000000
                    0x0000000000000000 0x0000000000000000  RW     0x10
      GNU_RELRO      0x0000000000002db0 0x0000000000003db0 0x0000000000003db0
                    0x0000000000000250 0x0000000000000250  R      0x1

    Соответствие раздел-сегмент:
      Сегмент Разделы...
      00     
      01     .interp 
      02     .interp .note.gnu.property .note.gnu.build-id .note.ABI-tag .gnu.hash .dynsym .dynstr .gnu.version .gnu.version_r .rela.dyn .rela.plt 
      03     .init .plt .plt.got .plt.sec .text .fini 
      04     .rodata .eh_frame_hdr .eh_frame 
      05     .init_array .fini_array .dynamic .got .data .bss 
      06     .dynamic 
      07     .note.gnu.property 
      08     .note.gnu.build-id .note.ABI-tag 
      09     .note.gnu.property 
      10     .eh_frame_hdr 
      11     
      12     .init_array .fini_array .dynamic .got 

Почему получилось 4 LOAD сегмента?  
Пишут, что обычно линковщик ld объединяет .text + .rodata ("Hello, World!" + "main address: %p\n") в один LOAD R+E сегмент. Плюс к секции .rodata прилепляется сами программные заголовоки заголовка ELF файла. Таким образом оптимизируются вызовы mmap для соответствующей разметки логической памяти под программу, но за счет безопасности (теоретически .rodata может стать исполняемой)...  
В новых линуксах так делать перестали. 
Более того, под программные заголовки делается отдельные сегмент, поэтому тут один LOAD R это константные данные из программы, другой - сам заголовок.  

For modern binutils(bfd) (v2.31.1+), use -Wl,-z,noseparate-code to reduce the number of LOAD-able segments back to 2. Так программа будет чуть быстрее стартовать (на 2 вызова mmap)

Таким образом, первая секция LOAD это сам заголовок файла, который очевидно лежит в ФАЙЛЕ в самом начале (по смещению 0x0) и будет копироваться в начало станицы памяти, которая будет выделена при запуске файла ОС.


## <a id='toc1_10_'></a>[Загрузка исполняемого файла](#toc0_)

Подготовить адресное пространство согласно описанию в файле

    аллоцировать память и настроить таблицы страниц;
    скопировать код и данные из файла в память;
    возможно создать стек отдельно 
        т.е. дополнительно к стеку того потока, который все это дело делает,
        т.е. один стек ядра, которое исполяет код приложения, один стек, который будет исп. приложением

"Прыгнуть" в userspace

    передать управление точке входа, указанной в файле;
    возможно, понизить уровень привилегий.

# <a id='toc2_'></a>[Библиотеки](#toc0_)

Виды библиотек:

    статические - становятся частью исполняемого файла;
        статическая линковка
    динамические - хранятся отдельно от исполняемого файла
        загружаются при запуске приложения или по требованию
        т.е. загружать в память тоже можно только 1 раз для всех, кому она потребуется
            Например:
                ОС один раз выделяет под библиотеку физические страницы памяти
                ОС пишет в них содержимое библиотеки
                ОС отображает физические страницы памяти где лежит библ. на логические адреса приложений
            *) но тогда все должны знать ее физический адрес, он должен быть закреплен и не перекрываться другими
               библиотеками. Т.е. пример плохой, но зато так приложения по идее знают где лежит библиотека, где у нее код,
               а где данные

    
## <a id='toc2_1_'></a>[Динамические библиотеки](#toc0_)

    Особенность динамических библиотек - могут быть загружены по разным адресам
    Библиотеки заранее не знают куда их поместит ОС, в отличие от исп.файла, где 
    формат самого файла (напр.ELF) задает смещения по файлу, где код, где данные и т.п.
    
        как код библиотеки обращается к своим коду и данным?
        как код приложения обращается к библиотеке?
        как код библиотек обращается к коду и данным других библиотек?
    

## <a id='toc2_2_'></a>[Компоновщик](#toc0_)

Компоновщик (linker, link editor) - программа, которая "связывает" бинарные файлы вместе и генерирует исполняемый файл

    в момент компиляции адреса функций/переменных могут быть не известны;
    компилятор просто оставляет "пустое место", а компоновщик записывает в него адрес.

## <a id='toc2_3_'></a>[Динамический компоновщик](#toc0_)

Так решается проблема того, что адреса функций/переменных (в памяти) из динамических библиотек заранее не известны  
NB адрес функции (переменной) это адрес места в памяти, где лежат инструкции процессора (данные)
Пока библиотеку не загрузили в память адреса эти нам неизвестны  

    компилятор/статический компоновщик оставляют "пустые места";
    динамический компоновщик должен записать в них адреса, после того как библиотека была загружена
        (т.е. некий дополнительный компоновщик, который работает уже при запуске приложения, т.е. это уже функционал ОС) 

## <a id='toc2_4_'></a>[Загрузка ELF файла с динамическими библиотеками](#toc0_)

    ELF файл загружается как обычно
        ищем Program Header-ы с типом PT_LOAD и загружаем их в память.
    Смотрим в Program Header с типом PT_INTERP
        там хранится (в виде строки) имя файла динамического компоновщика;
        ОС загружает его (как обычное приложение) в память в дополнение к программе.
    Передаем управление динамическому компоновщику (а не в точку входа изначального приложения).

*) динамический компоновщик не является частью ядра ОС (в Линукс и многих других ОС), но очень близок. Поэтому хоть это не по теме непосредственно ОС, но важный момент их работы. 

## <a id='toc2_5_'></a>[Поиск динамических библиотек](#toc0_)

Исполняемый файл должен хранить информацию о динамических библиотеках

    например, ELF Program Header с типом PT_DYNAMIC указывает, где в файле хранится эта информация;
    динамический компоновщик загружает все требуемые зависимости в память (или проверяет, что они загружены ранее).

## <a id='toc2_6_'></a>[Редактирование связей](#toc0_)

Исполняемый файл и динамические библиотеки хранят список обращений к внешним сущностям

    вызовы функций из других (и не только) библиотек;
    обращения к переменным (и не только) из других библиотек;
    динамический компоновщик находит адреса и записывает их в определенные места в памяти.

*) При запуске приложений эффективность динамического компоновщика влияет на скорость запуска.

# <a id='toc3_'></a>[Как это работает в случае ELF файлов](#toc0_)

## <a id='toc3_1_'></a>[GOT (для переменных)](#toc0_)

ELF формат использует Global Offset Table (GOT)

    код, обращающийся к переменной, знает относительный адрес GOT и номер записи в ней,соответствующей этой переменной;
    компилятор генерирует код, который берет адрес из GOT;
    динамический компоновщик записывает в GOT правильные адреса при загрузке.

Т.о. у каждой библиотеки свой GOT. При компиляции на все переменные в коде пишется GOT+offset. При загрузке в память приложения, использующего эту библиотеку на обращения из приложения к переменным из библиотеки пишется что-то типа GOT\[i\], с правильным адресом GOT, по которому получается нужное место в памяти, где лежат переменные библиотеки. Т.е. для каждой переменной при динамической линковке заполняется 1 запись в GOT, это быстро. При статической компоновке надо было бы в каждое место, где используется переменая библиотеки вписывать адрес объекта в памяти (компоновка чуть дольше, код чуть быстрее). 

## <a id='toc3_2_'></a>[PLT (для процедур/функций)](#toc0_)

ELF формат также использует Procedure Linkage Table(PLT)

    код обращающийся к функции знает относительный адрес PLT и номер "заглушки" в ней,соответствующей этой переменной;
    компилятор генерирует код, который вызывает "заглушку" из PLT вместо реальной функции;
    динамический компоновщик может изменять PLT, а может изменять GOT, к которой "заглушка" из PLT обращается.

Т.е. идея полностью такая же как GOT для переменных, кроме того, что компоновщик из соображений эффективности может оптимизировать вызовы библиотечного кода, когда надо делая такой прокси-вызов заглушки, которая вызовет целевой код, когда надо - просто передавая управление по адресу, типа как по запись в GOT...

Замечания: 

    использование GOT позволяет не менять загруженный код библиотек в адресном пространстве процесса, перебивая там 
    адреса внешних сущностей. А значит, соответствующие страницы в памяти можно разделять с другими процессами. 

    скорость и оптимальность работы дин.компоновщика важны, особенно если учитывать, что все эти манипуляции должны
    выполняться во время работы программы, а не при компиляции;

    на ubuntu 18.04 все исполняемые файлы как пользовательские, так  и системные имеют тип "DYN (Shared object file)"
    Зачем? Возможно, это связано с безопасностью. Если большую часть кода бинарного файла можно загружать где угодно, 
    то можно этот процесс рандомизировать и сделать немного тяжелее некоторые эксплойты;

    ВАЖНО понимать! Выше речь только о КОДЕ библиотек (локальные переменные и функции). Данные библиотек (т.е. 
    глобальные и статические переменные) отображаются на отдельные, различные для каждого процесса физические страницы,
    чтобы у каждого процесса они были свои (так работает по умолчанию). Т.е. разделяется (share) только код, но не 
    данные

# <a id='toc4_'></a>[Системные вызовы](#toc0_)

    Интерфейс системных вызовов
    Снова о прерываниях
    Програмные прерывания и дескрипторы IDT
    Дескриптор IDT в x86: селектор
    Переключение стеков
    Task State Segment x86
    Регистр TR в x86
    Резюме: системные вызовы, переключение стеков и TSS вместе
    Первый переход в userspace

## <a id='toc4_1_'></a>[Системные вызовы](#toc0_)

Системные вызовы - интерфейс между userspace и ядром ОС  
Память kernelspace защищена от непривилегированного доступа, поэтому:

    пользовательский код не имеет достаточно привилегий, чтобы вызывать код ядра как обычные функции;
    системный вызов сопровождается повышением привилегий;
    возврат из системного вызова сопровождается понижением привилегий.

## <a id='toc4_2_'></a>[Реализация системных вызовов](#toc0_)

Как реализовать интерфейс системных вызовов?
    
    способ, как обычно, зависит от архитектуры;
    например, в x86 существуют инструкции syscall и sysenter*;
    но мы посмотрим на другой вариант (более старый).

\*)  самый простой 32-битный вариант (более медленная работа)

    в регистр EAX загружается номер функции ядра
    параметры (зависящие от функции) загружаются в регистры EBX (первый) ECX EDX ESI EDI EBP (шестой)
        *) у реальных системных функций больше 5 параметров не встречается
    осуществляется вызов прерывания: int 0x80 (128)

    ; Linux 32-bit
    mov	eax,1		; linux system call number (sys_exit)
    int	0x80		; call kernel

\*\*) 32-bit инструкция sysenter в х86 (более быстрая (в 2-3 раза), т.к. использует специальные MSR-регистры для организации возврата и др.) + sysexit сделает сама ОС при возврате

    mov eax,function_number
    mov ebx,param_1 ; если есть
    mov ecx,param_2 ; если есть
    mov edx,param_3 ; если есть
    mov esi,param_4 ; если есть
    mov edi,param_5 ; если есть
    push .ret       ; адрес возврата
    push ecx        ; сохранение этих регистров требуется системой
    push edx
    push ebp
    mov ebp,esp
    sysenter        ; вызов!
    .ret:
    
\*\*\*) 64-битный код (x64 и x32 ABI, архитектура x86-64)  
Здесь всё несколько проще (за исключением странной очерёдности использования регистров для передачи параметров).

    1. В регистр RAX загружается номер функции.
    2. Параметры (зависящие от функции) загружаются в регистры RDI (первый) RSI RDX R10 R8 R9 (шестой).
    3. Выполняется инструкция syscall (не путайте с sysenter – это две разные инструкции!)

    mov rax,function_number
    mov rdi,param_1 ; если есть
    mov rsi,param_2 ; если есть
    mov rdx,param_3 ; если есть
    mov r10,param_4 ; если есть
    mov r8,param_5  ; если есть
    mov r9,param_6  ; если есть
    syscall         ; вызов! (регистры RCX и R11 будут уничтожены)
    
    ; если установлен CONFIG_IA32_EMULATION, то в линукс х64 будет также работать int 0x80 для 64-битных процессов

В современных ОС Линух в основном используется метод vDSO (Virtual Dynamic Shared Object) для системных вызовов. Который сделан на основе уже не используемого (из-за уязвимости) метода vsyscall. 

vsyscall  

Т.к. переключение контекста это долго, сделали типа разделяемой библиотеки. Для этого было зарезервировано 8 мб памяти для отображения пространства ядра в пространство пользователя. Для вызова генерировалось исключение PageFault, по адресу откуда оно пришло становилось понятно, что это просто сигнал для системного вызова и управление передавалось этой "разделяемой библиотеке", где лежали основные наиболее часто используемые системные функции ядра. Т.к. библиотека одна и лежит в одном месте, это уязвимость, заранее известно куда бить.

vDSO  
Это linux-vdso.so.1, где сохранены адреса основных системных функций и которая подключается (линкуется) к каждому пользовательскому приложению. В памяти размещается по технологии Address space layout randomization (ASLR) - при использовании которой случайным образом изменяется расположение в адресном пространстве процесса важных структур данных, а именно образов исполняемого файла, подгружаемых библиотек, кучи и стека. Внедрена в 2001 год в ядре Линукс, в 2007 добавилась в Виндоуз и Макос (в 2017 году показали, как и ее (ASLR) можно поломать).  
Этот способ сист. вызовов в 5-10 раз быстрее int 0x80.  

Для использования ASLR исполняемые файлы требуется собирать со специальными флагами. В результате в коде не будут использоваться постоянные адреса, но при этом:

    увеличится размер кода исполняемых файлов;
    увеличится время загрузки в память каждого исполняемого файла;
    возникнет дополнительная несовместимость с ПО и библиотеками, разработанным под версии ОС без ASLR.

Кроме того, защиту можно обойти через истощение памяти* или методы Heap Spray.   
\* забить доступную память процесса, освободить минимум и тогда ОС не останется маневра для рандомизации. 

## <a id='toc4_3_'></a>[И сново о прерываниях...](#toc0_)

Что происходит, если обработчик прерывания прерывает пользовательский код? 

    вызывается обработчик прерывания - а это код ядра;
    обработчик прерывания выполняется уже в привилегированном режиме (он в памяти ядра, и конечно должен иметь к нему доступ).

## <a id='toc4_4_'></a>[Программные прерывания](#toc0_)

Прерывания можно вызывать программно (и не только сделав ошибку)

    например, в x86 для этого существует специальная инструкция int, номер прерывания - параметр инструкции;
    выберем запись в IDT и будем использовать ее для системных вызовов (обычно выбирается 0х80).

## <a id='toc4_5_'></a>[Программные прерывания в x86](#toc0_)

С помощью инструкции int в x86 можно генерировать прерывание с любым номером
    
    в том числе и соответствующие исключениям;
    в том числе и соответствующие аппаратным прерываниям;
    приложения могут натворить бед, если разрешить им генерировать прерывания как попало.
    идея в том, чтобы пользовательским приложениям запретить генерацию любых прерываний, кроме системных вызовов


## <a id='toc4_6_'></a>[Дескриптор IDT, поле DPL (desrciptor privilege level)](#toc0_)

Таблица векторов прерываний (англ. Interrupt Descriptor Table, IDT) используется в архитектуре x86 и служит для определения корректного ответа на прерывания и исключения.   

    В х86 адрес в физической памяти и размер таблицы прерываний определяется 48-битным регистром IDTR  
    В реальном режиме элементом IDT является 32-битный FAR-адрес обработчика прерывания.  
    В защищённом режиме элементом IDT является шлюз прерывания длиной 8 байт, содержащий сегментный (логический) 
    адрес обработчика прерывания, права доступа и др.  
    В длинном режиме размер дескриптора прерывания увеличен до 16 байт.   

В дескрипторной таблице прерываний IDT могут содержаться шлю­зы трех типов:

    ·      шлюз ловушки;
    ·      шлюз прерывания;
    ·      шлюз задачи. 
    Физически микропроцессор отличает шлюзы по со­держимому полей. 

DPL дескриптора системного вызова выставляем в 3

    благодаря чему непривилегированный код может генерировать это прерывание.
    DPL всех остальных дескрипторов выставляем в 0 (если такое прерывание попробует сгенерить приложение, то исключение -> выход).

### <a id='toc4_6_1_'></a>[Тут нужно отметить](#toc0_)

Уровень привелегий текущего исполняемого кода определяется 2 посл. битами в регистре CS.  
В структуре IDT вторая запись всегда Segment Selector (первая запись всегда - обработчик zerodivision, остальные как ОС сделает).   
При вызове обработчика прерывания, текущее значение CS (с низкой привилегией) сохраняется на стеке, Segment Selector из IDT копируется в рег. CS, повышая уровень привилегий кода таким образом, затем при вызове iretq восстанавливается со стека исходное значение CS.  

*) В защищённом режиме процессора адресное пространство задачи делится на сегменты различных размеров с различными базами. Для определения базы и размера сегментов служат дескрипторы сегментов, хранящиеся в дескрипторных таблицах (GDT и LDT). Сегменты (в GDT) могут указывать на одну и ту же область, тогда они являются псевдонимами (алиасными от англ. Alias). Сама таблица GDT лежит в памяти по физ.адресу из регистра GDTR base и занимает в ней GDTR limit байт.




## <a id='toc4_7_'></a>[Стек обработчика прерывания](#toc0_)

При вызове обработчика на стек сохраняются адрес возврата и прочее

    на какой стек все это будет сохранено?
    не хочется использовать стек непривилегированногокода
        там может быть не достаточно места;
        пользовательский код может делать со своим стеком все что угодно (вообще не использовать, а например, с регистрами работать).

## <a id='toc4_8_'></a>[Отдельный стек для ядра](#toc0_)

Мы хотим использовать отдельный стек для ядра и отдельный для userspace

    например, в Linux для каждого потока создается стек ядра, т. е. у каждого потока есть 2 стека;
    при прерываниях и системных вызовах происходит переключение на стек ядра этого потока.

## <a id='toc4_9_'></a>[Task State Segment (х86)](#toc0_)

TSS (Task State Segment) - структура, которая хранит значение указателя стека, который будет загружен в RSP (когда обработчик прерывания задействуется в непривилегированном коде).

    ранее (в 32-битном режиме) могла быть использована для хранения состояния потока (контекст), отсюда такое название.

"Указание" на TSS хранится в специальном регистре TR

    инструкция LTR записывает значение в TR, а инструкция STR читает;
    для использования TSS необходимо завести специальный дескриптор в GDT
        Base и Limit хранят логический адрес и размер TSS;
        специальные флаги показывают, что эта запись GDT является именно описанием TSS;
        селектор этого дескриптора сохраняется в TR.

*) в GDT хранятся записи разных типов и разных форматов, не обязательно 32 битные

Простой вариант использования TSS:
    
    создаем по TSS на каждое ядро процессора - один раз, при инициализации ядра ОС
        *) например структура TSS в виде глобальной переменной в коде ядра
           один раз под каждую TSS сделать дескриптор в GDT
    при переключении потоков подменяем указатель стека в TSS.

## <a id='toc4_10_'></a>[Резюме](#toc0_)

    Подготовить дескриптор IDT, который будет использоваться для системных вызовов.
    Создать TSS:
        создать дескриптор, описывающий TSS, в GDT;
        загрузить селектор, ссылающийся на дескриптор, в TR.
    Не забывать подменять указатель стека в TSS при переключении потоков.

Ровно так сделано в учебном примере ОС.
    

## <a id='toc4_11_'></a>["Прыжок" в userspace](#toc0_)

Как передать управление в userspace в первый раз, после того как ядро ОС загрузило его в память (в т.ч. если это приложение - динамический линковщик для другого пользовательского приложения), понизив уровень привилегий с уровня ядра?

    инструкция iretq завершает обработчик прерывания и передает управление, возможно, понизив уровень привилегий;
    инструкция iretq берет свои параметры со стека - поэтому подготовим стек в формате, нужном iretq и вызовем iretq

Формат этого стека:

    SS      + 32        // сюда и в CS значение, где младшие 2 бита равны 3 (10b)
    RSP     +24         // указатель стека приложения
    RFLAGS  +16         // флаги приложения
    CS      +8          // сюда и в SS значение, где младшие 2 бита равны 3 (10b)
    RIP     RSP + 0     // управление должно быть передано сюда (кладем точку входа)

# <a id='toc5_'></a>[Средства управления процессами](#toc0_)

    Какие сервисы может предоставлять ОС
    Создание процессов в Unix: fork
    fork и многопоточность
    Уничтожение процессов в Unix: exit и waitpid
    Уничтожение процессов в Unix: потоки и Zombie
    Запуск исполняемых файлов в Unix: exec*
    exec* и многопоточность

## <a id='toc5_1_'></a>[Системные вызовы](#toc0_)

Какие бывают системные вызовы? Зачем польз.прогр. обращаться к функциям ОС в принципе?

Cервисы ОС:
    
    работа с файловыми системами;
    работа со временем (текущее время, нотификации и прочее)
    создание, завершение и управление процессами (fork, wake etc);
    взаимодействие с другими процессами (у каждого процесса своя память по умолчанию, без ОС они друг друга не увидят).

## <a id='toc5_2_'></a>[Создание процессов](#toc0_)

Для создания процессов в Unix-like системах используется вызов fork:

    новый процесс является почти точной копией родителя (переменные такие же, но значения каждый сам определяет);
    вызывает fork один поток, а возвращаются из fork уже два потока в двух разных процессах.

Что если в процессе несколько потоков, и один из них вызвал fork?

    в новом процессе будет только один поток 
        *fork скопирует только один (активный) контектс потока;
    подумайте о блокировках в новом процессе 
        *например, один поток захватит блокировку, 
        второй поток сначала сделает форк, потом встал на ожидание разблокировки,
        первый поток ее отпустил, второй поток штатно продолжил работу
        *в это время форкнутый поток (копируется только он со всеми данными, в т.ч. блокировками (которые просто
         структуры в памяти)), будет висеть заблокированный, т.к. того, кто отпустит ЕГО блокировку - не существует

## <a id='toc5_3_'></a>[Уничтожение процессов](#toc0_)

Уничтожение процессов состоит из двух частей (обязательно наличие как самого завершаемого процесса, так и его родителя):

    один из потоков в процессе должен вызвать exit
        exit принимает целочисленный код как аргумент - код возврата.
    родительский процесс (родной или некий приемный) должен дождаться завершения процесса, используя waitpid/wait
        wait/waitpid могут вернуть код возврата, переданныйв exit.

    Что если в контексте процесса работают несколько потоков?
        exit уничтожает процесс совсеми его потоками *
            *) как убивать поток зависит от того, как вы поток создали, и для этого не всегда нужен системный вызов.
            если мы говорим о linux и NPTL (aka реализация POSIX threads под linux), то там используется системный 
            вызов exit, в то время как Сишная функция exit вызывает exit_group, чтобы завершить все потоки процесса.

    Что если родитель был уничтожен раньше ребенка?
        другой процесс становится родителем **.
            **)новый родитель оставшегося ребенка определяется системно-зависимым способом, но обычно им становится
            процесс, который ведет себя как «идеальный родитель» — тщательно делает waitpid на всех своих детях. 
            Соответственно, от зомби очень легко избавиться — надо просто уничтожить их нерадивого родителя, тогда их
            передадут правильному «приемному», который с ними разберется
            
    Что если не вызвать waitpid/wait?
        процесс будет находится в состоянии Zombie, пока родитель не вызовет wait/waitpid.


## <a id='toc5_4_'></a>[Запуск исполняемых файлов](#toc0_)

Для запуска исполняемого файла используется один из вызовов exec*:

    при вызове exec* ядро ОС "заменяет" старое адресное пространство процесса новым;
    передает управление точке входа исполняемого файла (или динамического компоновщика).

    Что если в процессе несколько потоков и один из них вызвал exec*?
        все другие потоки будут уничтожены.

## <a id='toc5_5_'></a>[Файловые дескрипторы](#toc0_)

В Unix все есть файл:
    
    некоторые ресурсы, предоставляемые ОС, имеют файловый интерфейс;
    файловый интерфейс: read/write/close.

Файловый дескриптор - некоторый идентификатор ресурса

    в Unix - это обычно просто целое число;
    0 - стандартный поток ввода,
    1 - стандартный поток вывода,
    2 - стандартный поток ошибок.

Способ получения дескриптора зависит от ресурса, которому он соответствует:

    для обычных файлов можно использовать open;
    для каналов (pipe-ов) используют pipe;
    есть много других функций, возвращающих файловый дескриптор.

## <a id='toc5_6_'></a>[Дублирование дескрипторов](#toc0_)

Иногда вам может потребоваться управлять значением файлового дескриптора
    
    например, чтобы перенаправлять стандартные потоки ввода/вывода в файл/из файла;
    для этого можно использовать вызов dup2.


# <a id='toc6_'></a>[IPC](#toc0_)

Виды IPC, которые мы НЕ БУДЕМ рассматривать
    
    Сигналы 
        (напр. alarm() - SIGALRM, kill() - SIGKILL)
    Семафоры
        (напр. Lock - семафор синхронизации??)
    Сокеты
        (обращения к сетевым сервисам ОС, мощная штука, позволяющая связывать процессы даже различных машинах по разными ОС)

Виды IPC, которые мы будем рассматривать

    Каналы (неименованные каналы).
    FIFO (именованные каналы).
    Сегменты разделяемой памяти.
    Ptrace.

## <a id='toc6_1_'></a>[Каналы (Pipes)](#toc0_)

Канал - односторонний (!) канал связи между процессами (или внутри процесса)

    создается вызовом pipe - вызов возвращает два "файловых дескриптора";
    через один можно писать, из другого можно читать;
    работа с pipe происходит почти как с обычным файлом: read/write/close.

## <a id='toc6_2_'></a>[Именованные каналы (FIFO)](#toc0_)

Почти как pipe, но у FIFO есть имя
    
    создается с помощью специальной функции mknod;
    работа с FIFO происходит почти как с обычным файлом: open/read/write/close;
    при попытке открыть FIFO поток по умолчанию блокируется, пока кто-нибудь не откроет "другой конец" FIFO.

## <a id='toc6_3_'></a>[Сегменты разделяемой памяти (по смыслу - как файл, только в RAM)](#toc0_)

Сегмент разделяемой памяти - участок памяти, к которому можно обращаться из нескольких процессов \*.  
Типа как потоки одного процесса, имеющие общее адресное пространство по определению.  
Ничего не мешает ОС выделить память и отобразить в адресное пространство разных процессов.

    создать сегмент можно с помощью shmget (выделить участок памяти);
    получить указатель на сегмент памяти можно спомощью shmat (shared memory attach) (отобразить на адресное
    пространство процесса - у каждого процесса разные значения указателя, отображаются на 
    одно место в физ.памяти);
    "закрыть" сегмент можно с помощью shmdt (shared memory detach).

*) или один и тот же процесс может обращаться после завершения и возобновления. ОС держит этот участок и после завершения приложения, потом опять можно подключиться и все сохраниться, прям как файл.  

    $ /home/user1/projects/stepik.org/OS/5.Userspace/shm/main -g            // прочитали - пусто

    $ /home/user1/projects/stepik.org/OS/5.Userspace/shm/main -p "Hello"    // записали
    $ /home/user1/projects/stepik.org/OS/5.Userspace/shm/main -g            // прочитали - сохранилось
    Hello


## <a id='toc6_4_'></a>[Process Trace](#toc0_)

ptrace не является IPC в привычном понимании:
    
    позволяет одному процессу следить за другим;
    позволяет одному процессу подсмотреть в память другого;
    позволяет одному процессу изменить память другого (или даже отдельные регистры).

Пример 1: типа аналоги strace - выводит портянку сисколов некоторого приложения  
Пример 2: тоже самое, но еще и с подменой аргументов сискола (органичение на подмену - только для процессов, которые сами и запустили, разрешив работу ptrace) - вывод стандартной команды ls заменяется на Hello, World ))

Мораль - это и есть основной интерфейс для работы дебаггеров - смотреть/менять переменны, расставлять точки останова и т.д. Все что они умеют делать это, грубо говоря, обертка для сискола ptrace 